<a href="https://colab.research.google.com/github/AntonioFialhoSN/Entrega_da_P2_RedesNeurais/blob/main/Atividade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Atividade: Callbacks e Schedulers em Redes Neurais

## Introdução

Nesta atividade você irá trabalhar com dois importantes recursos no treinamento de redes neurais:

- **Callbacks**: funções que atuam durante o treinamento para monitorar e ajustar o processo.
- **Schedulers de Taxa de Aprendizado**: funções que controlam a variação da taxa de aprendizado durante o treinamento.

Vamos utilizar o dataset MNIST e um modelo de rede neural simples (MLP).

## 1. Importação de Bibliotecas

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, callbacks
import matplotlib.pyplot as plt

## 2. Carregamento e Pré-processamento do Dataset

In [ ]:
# Carregar MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalizar os dados
x_train = x_train.reshape(-1, 28*28).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28*28).astype('float32') / 255.0

## 3. Definição do Modelo

In [ ]:
# Cria uma rede neural simples com uma camada oculta
def create_model():
    return tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(784,)),
        layers.Dense(10, activation='softmax')
    ])

model = create_model()
model.compile(optimizer=Adam(0.1),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

## 4. EarlyStopping

Agora vamos implementar um callback EarlyStopping, que interrompe o treinamento se a validação não melhorar após algumas épocas.\

In [ ]:
early_stop = callbacks.EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True
)

**Pergunta:** Qual o efeito de `patience=3` e `restore_best_weights=True`?

## 5. Implementação de Step Decay

Agora vamos criar um scheduler com redução abrupta (step decay).

In [ ]:
def step_decay(epoch, initial_lr=0.01, decay_rate=0.1, decay_steps=10):
    return initial_lr * (decay_rate ** (epoch // decay_steps))

step_scheduler = callbacks.LearningRateScheduler(step_decay)

## 6. Treinamento com Step Decay

In [ ]:
history_step = model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    callbacks=[early_stop, step_scheduler]
)

## 7. Implementação de Exponential Decay

In [ ]:
def exp_decay(epoch, initial_lr=0.1, k=0.05):
    return initial_lr * math.exp(-k * epoch)

exp_scheduler = callbacks.LearningRateScheduler(exponential_decay)

## 8. Treinamento com Exponential Decay

Antes de treinar novamente, precisamos resetar o modelo

In [ ]:
model = create_model()
model.compile(optimizer=Adam(0.1),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history_exp = model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    callbacks=[early_stop, exp_scheduler]
)

## 9. Comparação Gráfica

Agora vamos comparar os dois métodos.

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history_step.history['val_loss'], label='Val Loss - Step Decay')
plt.plot(history_exp.history['val_loss'], label='Val Loss - Exp Decay')
plt.legend()
plt.title('Comparação da Validação Loss')

plt.subplot(1,2,2)
plt.plot(history_step.history['val_accuracy'], label='Val Acc - Step Decay')
plt.plot(history_exp.history['val_accuracy'], label='Val Acc - Exp Decay')
plt.legend()
plt.title('Comparação da Validação Accuracy')

plt.show()

## 10. Desafio

- Modifique os valores de `initial_lr` e `decay_rate` no `decay_steps` e observe o impacto.
- Modifique o valor de `k` no `exponential_decay`.
- Experimente remover o `EarlyStopping` e observe como o modelo se comporta.

Depois de fazer os experimentos verifique e e faça uma descrição dos resultados obtidos